In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.set_option('plotting.backend', 'plotly')
from datetime import date, datetime as dt
import os, re
import plotly.colors
import plotly.graph_objects as go, plotly.express as px, plotly.figure_factory as ff
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots

from statsmodels.stats.proportion import proportion_confint

from tennis_utils.player import TennisDataLoader, TennisPlayerDataLoader, TennisPlayer, TennisPlayerRenderer

def get_player_name(name):

    return '. '.join(['.'.join([e[0] for e in name.split(' ')[:-1]]), name.split(' ')[-1]])


In [8]:

player_name = 'Diego Sebastian Schwartzman'
data_path = os.getcwd()+'/data'

tdl = TennisDataLoader(data_path)
matches_df, players_df = tdl.matches, tdl.players

tpdl = TennisPlayerDataLoader(player_name, matches_df, players_df)

# # Subset selected player matches data
player_matches = tpdl.player_matches #matches_df[matches_df['player_name'] == player_name]
player_details = tpdl.player_details #players_df[players_df['player_name']==player_name]
# player_rank = tpdl.player_rank

# tp = TennisPlayerRenderer(player_name=player_name, 
#                           player_matches=player_matches, 
#                           player_rank=player_rank, 
#                           player_details=player_details)


106043


In [12]:
matches_df.loc[lambda x: x['id']=='106043']

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,id,seed,entry,...,perc_returnWon,perc_servePointsWon,tbPlayed,tbWon,decidingSetPlayed,winner,result,year,decidingSetWon,secondIn
105514,2010-6208,Buenos Aires2 CH,Clay,32,C,2010-11-22,14,106043,nan,WC,...,0.426966,0.540541,0,0,1,0,Lost,2010,0.0,30
108676,2011-2121,Manta CH,Hard,32,C,2011-07-18,3,106043,nan,nan,...,0.388235,0.557895,0,0,1,0,Lost,2011,0.0,33
111118,2011-3406,Santiago CH,Clay,32,C,2011-03-07,13,106043,nan,Q,...,0.439394,0.454545,0,0,0,0,Lost,2011,NaN,17
112197,2011-3627,Todi CH,Clay,32,C,2011-09-12,13,106043,nan,nan,...,0.573770,0.600000,0,0,0,1,Won,2011,NaN,30
112216,2011-3627,Todi CH,Clay,32,C,2011-09-12,23,106043,nan,nan,...,0.428571,0.564356,2,1,1,0,Lost,2011,0.0,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300832,2021-580,Australian Open,Hard,128,G,2021-02-08,203,106043,8.0,nan,...,0.311111,0.546875,0,0,0,0,Lost,2021,NaN,15
301443,2021-8888,Atp Cup,Hard,12,A,2021-02-02,272,106043,nan,nan,...,0.518987,0.620000,1,0,1,1,Won,2021,1.0,32
301450,2021-8888,Atp Cup,Hard,12,A,2021-02-02,276,106043,nan,nan,...,0.406250,0.500000,0,0,0,0,Lost,2021,NaN,29
301705,2021-9158,Cordoba,Clay,32,A,2021-02-22,293,106043,1.0,nan,...,0.583333,0.633333,0,0,0,1,Won,2021,NaN,22


In [11]:
matches_df.loc[lambda x: x['player_name'].str.contains('Schwartzman')]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,id,seed,entry,...,perc_returnWon,perc_servePointsWon,tbPlayed,tbWon,decidingSetPlayed,winner,result,year,decidingSetWon,secondIn
105514,2010-6208,Buenos Aires2 CH,Clay,32,C,2010-11-22,14,106043,nan,WC,...,0.426966,0.540541,0,0,1,0,Lost,2010,0.0,30
108676,2011-2121,Manta CH,Hard,32,C,2011-07-18,3,106043,nan,nan,...,0.388235,0.557895,0,0,1,0,Lost,2011,0.0,33
111118,2011-3406,Santiago CH,Clay,32,C,2011-03-07,13,106043,nan,Q,...,0.439394,0.454545,0,0,0,0,Lost,2011,NaN,17
112197,2011-3627,Todi CH,Clay,32,C,2011-09-12,13,106043,nan,nan,...,0.573770,0.600000,0,0,0,1,Won,2011,NaN,30
112216,2011-3627,Todi CH,Clay,32,C,2011-09-12,23,106043,nan,nan,...,0.428571,0.564356,2,1,1,0,Lost,2011,0.0,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300832,2021-580,Australian Open,Hard,128,G,2021-02-08,203,106043,8.0,nan,...,0.311111,0.546875,0,0,0,0,Lost,2021,NaN,15
301443,2021-8888,Atp Cup,Hard,12,A,2021-02-02,272,106043,nan,nan,...,0.518987,0.620000,1,0,1,1,Won,2021,1.0,32
301450,2021-8888,Atp Cup,Hard,12,A,2021-02-02,276,106043,nan,nan,...,0.406250,0.500000,0,0,0,0,Lost,2021,NaN,29
301705,2021-9158,Cordoba,Clay,32,A,2021-02-22,293,106043,1.0,nan,...,0.583333,0.633333,0,0,0,1,Won,2021,NaN,22


In [38]:
import yaml# import load, Loader, dump, Dumper

with open(os.getcwd() + '/config.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    data = yaml.load(file, Loader=yaml.Loader)

details_mapping = data['details_mapping']

In [3]:
"""
https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval
https://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportion_confint.html
https://www.ucl.ac.uk/english-usage/staff/sean/resources/binomialpoisson.pdf
"""

'\nhttps://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval\nhttps://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportion_confint.html\nhttps://www.ucl.ac.uk/english-usage/staff/sean/resources/binomialpoisson.pdf\n'